# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Mы будем работать с файлом, предоставленным банком, под названием 'data', который мы загрузим с помощью pandas.

В этих данных встречаются столбцы со **значениями переменных**, чьи названия расшифровываются следующим образом:
- `children` = количество детей в семье
- `days_employed` = общий трудовой стаж в днях
- `dob_years` = возраст клиента в годах
- `education` = уровень образования клиента
- `education_id` = идентификатор уровня образования
- `family_status` = семейное положение
- `family_status_id` = идентификатор семейного положения
- `gender` = пол клиента
- `income_type` = тип занятости
- `debt` = имел ли задолженность по возврату кредитов
- `total_income` = ежемесячный доход
- `purpose` = цель получения кредита


**В процессе иследования мы будем искать ответы на вопросы, поставленные нам банком:**
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?


**План нашего исследования:**
1. Оценка качества данных, просмотр информации о столбцах
2. Предобработка данных (разбираемся со странными, нулевыми и пропущеными значениями, дубликатами, итд.)
3. Группировка данных по уровню дохода
4. Формирование сводных таблиц, поиск закономерностей и зависимостей
5. Финальный вывод, рекоммендации для банка, заполнение чек-листа

**Оглавление:**

1. [Открытие файла, изучение данных](#introduction)
    * [Просмотр характеристики данных](#review)
    * [Выводы насчёт `days_employed` и `total_income`](#days_income_review)
    * [Выводы насчёт `children`](#children_review)
    * [Выводы насчёт `dob_years`](#age_review)
    * [Выводы насчёт `education` и `education_id`](#education_review)
    * [Выводы насчёт `family_status` и `family_status_id`](#family_review)
    * [Выводы насчёт `gender`](#gender_review)
    * [Выводы насчёт `income_type` и `debt`](#income_type&debt_review)
    * [Выводы насчёт `purpose`](#purpose_review)


2. [Предобработка данных](#datawork)
    * [Обработка подозрительно высоких и отрицательных трудовых стажей](#datawork1)
    * [Обработка пропусков в столбцах `days_employed` и `total_income`](#datawork2)
    * [Замена типа данных столбцов `days_employed` и `total_income`](#datawork3)
    * [Исправление значений в столбце с детьми](#datawork4)
    * [Работа с нулевым значением возраста](#datawork5)
        * [Разбираемся с нюансом в `dob_years`, непредвиденным до замены нулевых значений](#datawork5.1)
    * [Замена больших букв](#datawork6)
    * [Удаление значения 'XNA' в столбце с полом клиентов](#datawork7)
    * [Обработка дубликатов](#datawork8)
        * [Автоматизированое удаление дубликатов](#datawork81)
        * [Ручной просмотр дубликатов в столбце `purpose`, определение основных значений с помощью подсчёта  `lemmas`](#datawork82)
    * [Лемматизация](#lemmas)
    * [Категоризация данных](#categorisation)
    

3. [Подведение итогов, ответы на вопросы](#results)


4. [Заключение - словари в данных:](#dictionaries)


5. [Общий вывод](#conclusion)


6. [Чек-лист готовности проекта](#checklist)
    

## Открытие файла, изучение данных<a id="introduction"></a>

### Просмотр характеристики данных<a id="review"></a>

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
from IPython.display import display
display(data.head(25))

# Для начала используется метод head(25), чтобы получить больше распечатаных значений - 
# иначе таблица выводится в сокращённом формате и мы не сможем сразу "на глаз" оценить качество данных на наличие дубликатов, пропусков итд.


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()
# Используем info() чтобы выявить столбцы с пропусками.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, есть пропуски в `days_employed` и в `total_income`, а ещё мы сразу замечаем, что в `days_employed` есть отрицательные значения, от которых тоже нужно будет избавляться.

### Выводы насчёт `days_employed` и `total_income`<a id="days_income_review"></a>

**Анализируем проблематичные данные в `days_employed` и `total_income`:**

In [3]:
# Расчитаем доли пропусков в столбцах 'days_employed' и 'total_income':
ratio_days_employed = data['days_employed'].isnull().sum() / (data['days_employed'].count() + data['days_employed'].isnull().sum())
ratio_total_income = data['total_income'].isnull().sum() / (data['total_income'].count() + data['total_income'].isnull().sum())
print('{:.2%} - доля пропусков в "days_employed"\n{:.2%} - доля пропусков в "total_income"'.format(ratio_days_employed, ratio_total_income))

10.10% - доля пропусков в "days_employed"
10.10% - доля пропусков в "total_income"


Видимо, около одной десятой доли клиентов не указали свой стаж и месячный доход, либо же произошёл сбой при создании таблицы. Это уже весьма значимая часть данных и мы не можем просто удалить эти строки. Давайте проверим их тип занятости - может это одни безработные, которые не имееют никакого дохода или стажа на момент заполнения анкеты в банке? Тогда бы мы могли заполнить пропуски просто значением '0'...

In [4]:
# Проверяем наличие пропусков в столбце со стажем по отношению к типу занятости:
a = 0
import math
type_vs_days = pd.DataFrame(columns=['days_employed'])
while a < len(data['days_employed']):
    if math.isnan(data.loc[a, 'days_employed']) == True:
        inc_type = data.loc[a, 'income_type']
        type_vs_days = type_vs_days.append({'days_employed': inc_type}, ignore_index=True)
    a +=1
null_days = pd.DataFrame(columns=['null_days'])
null_days['null_days'] = type_vs_days['days_employed'].value_counts()
null_days

,null_days
сотрудник,1105
компаньон,508
пенсионер,413
госслужащий,147
предприниматель,1


In [5]:
# Повторяем этот-же процесс для столбца с доходом:
b = 0
type_vs_income = pd.DataFrame(columns=['total_income'])
while b < len(data['total_income']):
    if math.isnan(data.loc[b, 'total_income']) == True:
        inc_type = data.loc[b, 'income_type']
        type_vs_income = type_vs_income.append({'total_income': inc_type}, ignore_index=True)
    b +=1
null_income = pd.DataFrame(columns=['null_income'])
null_income['null_income'] = type_vs_income['total_income'].value_counts()
null_income

,null_income
сотрудник,1105
компаньон,508
пенсионер,413
госслужащий,147
предприниматель,1


Заполнить пропуски '0' не получится, ведь у пенсионеров не может быть нулевой стаж (в РФ наличие трудового стажа - один из критериев получения пенсии), также как и у работающего сотрудника или госслужащего не может быть нулевой доход.



In [6]:
# Посчитаем долю пропусков по отношению к общему количеству данных по виду деятельности:
data_by_type = data.groupby('income_type').agg({'income_type': ['count']})
data_by_type.columns = data_by_type.columns.droplevel(0)

# Доля пропусков в столбце со стажем:
data_by_type['null_days'] = null_days['null_days']
data_by_type['ratio_null_days'] = null_days['null_days']/data_by_type['count'] * 100

# Доля пропусков в столбце с доходом:
data_by_type['null_income'] = null_income['null_income']
data_by_type['ratio_null_income'] = null_income['null_income']/data_by_type['count'] * 100
data_by_type

,count,null_days,ratio_null_days,null_income,ratio_null_income
income_type,,,,,
безработный,2,NaN,NaN,NaN,NaN
в декрете,1,NaN,NaN,NaN,NaN
госслужащий,1459,147.0,10.075394,147.0,10.075394
компаньон,5085,508.0,9.990167,508.0,9.990167
пенсионер,3856,413.0,10.710581,413.0,10.710581
предприниматель,2,1.0,50.000000,1.0,50.000000
сотрудник,11119,1105.0,9.937944,1105.0,9.937944
студент,1,NaN,NaN,NaN,NaN


Доля пропусков в большинстве случаев - около 10%, их мы можем заменить с помощью среднего или медианы из `non-NaN` значений. Вопрос возникает с предпринемателем - их всего 2 и одно из значений пропущено. Придётся заменить единственным имеющимся непропущенным значением, но надо иметь ввиду, что это одно значение - нерепрезентативная выборка!


Прежде чем делать выводы по столбцам `days_employed` и `total_income` не дурно проверить, нет ли в наших данных неправдоподобно-больших стажей. Четвёртая строка со значением `340266.072047` дней (или 932 года!) выглядит проблематично.

In [7]:
# Добавим вспомогательный столбец 'years' с общим стажем переведённым в года, отсортируем его по убыванию и выведем первые 5 значений.
data['years'] = data['days_employed'] / 365
data.sort_values(by = 'years', ascending = False).head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью,1100.699727
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования,1100.591265
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем,1100.479708
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили,1100.477991
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба,1100.448904


Даже у пенционеров не может быть стаж 1100 лет. Допустим, что средний пенсионный возраст с учётом пола, вида деятельности и пенсионных реформ - 60 лет, а клиенты банка работают с 18 лет, или как только достигают совершеннолетия. Значит, максимальньный трудовой стаж (даже у пенсионеров) - 42 года. Посмотрим, как распределены эти сверхвысокие стажи по отношению к виду деятельности:


In [8]:
extreme_years = pd.DataFrame(columns=['extreme_years'])
# Важно не забыть, что значения 'years', также как и 'days_employed' могут быть отрицательными, поэтому для сравнения используем функцию 'abs()'.
extreme_years['extreme_years'] = data.loc[abs(data['years']) > 42]['income_type'].value_counts()

# Сразу добавим эти значения в нашу таблицу 'data_by_type':
data_by_type['extreme_years'] = extreme_years['extreme_years']

# Заодним проверим распределение отрицательных значений, может они присутсвуют только в отдельных категориях типа занятости и совпадают с наличием сверхвысоких стажей?
negative_years = pd.DataFrame(columns=['negative_years'])
negative_years['negative_years'] = data.loc[data['years'] < 0]['income_type'].value_counts()
data_by_type['negative_years'] = negative_years['negative_years']

data_by_type

,count,null_days,ratio_null_days,null_income,ratio_null_income,extreme_years,negative_years
income_type,,,,,,,
безработный,2,NaN,NaN,NaN,NaN,2.0,NaN
в декрете,1,NaN,NaN,NaN,NaN,NaN,1.0
госслужащий,1459,147.0,10.075394,147.0,10.075394,NaN,1312.0
компаньон,5085,508.0,9.990167,508.0,9.990167,3.0,4577.0
пенсионер,3856,413.0,10.710581,413.0,10.710581,3443.0,NaN
предприниматель,2,1.0,50.000000,1.0,50.000000,NaN,1.0
сотрудник,11119,1105.0,9.937944,1105.0,9.937944,7.0,10014.0
студент,1,NaN,NaN,NaN,NaN,NaN,1.0


Отрицательные стажи не совпадают со сверхвысокими данными - это два отдельных вопроса.
Но мы замечаем проблему у пенсионеров и безработных: у первых `count = extreme_years`, а у вторых `count = null + extreme_years` (3856 - 3443 = 413). Так как у нас нет других, более адекватных значений стажей этих двух категорий, придётся использовать то что имеем, так как пропуски всё рабно нужно заполнить, а удалив всех пенсионеров и безработных мы рискуем потерять значимую часть данных.

**Финальные выводы о данных в `days_employed` и `total_income`:**

1. Пропуски и отрицательные или слишком большие значения в столбце `days_employed` появились либо из-за человеческого фактора, либо из-за сбоя в банковской системе. Сначала мы заменим все отрицательные числа их модулем, а пропущеные значения мы будем заменять на среднее значение стажа для данного вида деятельности. Также нужно перебести тип данных `float64` (= вещественные числа) на `int64` (= целые числа).
2. Пропуски в столбце `total_income` могли возникнуть по аналогичным причинам, заменять мы их также будем аналогичным способом. Тип данных этого столбца также нужно поменять с `float64` на `int64`.




### Выводы насчёт `children`<a id="children_review"></a>

In [9]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значения 20 детей и -1 ребёнок выглядят подозрительно.

In [10]:
#Расчитываем долю странных значений '-1' и '20' в столбце 'children':
print('{:.3%} - доля "-1" ребёнка'.format(47 / data['children'].count()))
print('{:.3%} - доля "20" детей'.format(76 / data['children'].count()))


0.218% - доля "-1" ребёнка
0.353% - доля "20" детей


**Причины возникновения и возможные пути замены странных данных:**

Величины '-1' и '20' скорее всего возникли из-за невнимательного ввода данных или из-за сбоя в банковской системы генерирования таблицы с данными. Мы их можем либо удалить как ошибочные, потому что их доля не значительна по сравнению с количеством данных в выборке, либо заменить их соответсвенно на значения '1' и '2', которые наверное подразумевались под странными числами '-1' и '20'.

### Выводы насчёт `dob_years`<a id="age_review"></a>

In [11]:
# Проверяем на наличие нулевых значений в столбце с возрастом:
data.loc[data['dob_years'] == 0]['dob_years'].value_counts()

0    101
Name: dob_years, dtype: int64

In [12]:
print('{:.3%} - доля нулевых значений возраста клиентов по отношению ко всей выборке'.format(101 / data['dob_years'].count()))

0.469% - доля нулевых значений возраста клиентов по отношению ко всей выборке


**Причины возникновения и возможные пути замены нулевых значений возраста:**

Здесь нет пропусков, но есть 101 нулевое значение, а ведь клиентам банка не может быть 0 лет! Полагаем, клиенты не очень внимательно заполняли банковскую анкету или, опять же, произошёл сбой в системе генерирования таблицы данных. Эти нулевые значения составляют не значительную часть всей выборки и мы их можем удалить, но можно их также попробовать заменить на среднее значение возраста для данного типа деятельности.


**Примечание:** эти нулевые значения припятсвуют нам в выявлении других нестандартных значений возраста (например, "восьмилетних госслужащих или пенсионеров"), нам нужно будет проверит столбец на наличие таких странных данных уже *после* удаления нулевых значений и провести соответсвующию предобработку.


### Выводы насчёт `education` и `education_id`<a id="education_review"></a>

In [13]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [14]:
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

**Причины возникновения и способ замены больших букв в значениях столбца `education`:**

Здесь нужно будет применить lower(), чтобы количества уникальных значений идентификаторов уровня образования совпадали с количествами уникальных значений названий уровня образования. Отличающийся регистр букв возможен из-за невнимательности, недостатка инструкций клиентам или из-за сбоев при обработке данных банком.

### Выводы насчёт `family_status` и `family_status_id`<a id="family_review"></a>

In [15]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [16]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

**Причины возникновения и способ замены больших букв в значениях столбца `family_status`:**

Эти два столбца совпадают, но нужно будет применить lower() к 'Не женат / не замужем' чтобы унифицировать `family_status`. Эта ошибка могла возникнуть из-за невнимательного заполнения анкеты клиентами или в процессе переписывания значений в банковскую базу данных.

### Выводы насчёт `gender`<a id="gender_review"></a>

In [17]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [18]:
#Расчитываем долю странного значения 'XNA' в столбце 'gender':
print('{:.3%}'.format(1 / data['gender'].count()))

0.005%


**Причины возникновения и возможные пути замены странного значения `XNA`:**

Один из клиентов не указал свой пол (`XNA` скорее всего должно означать тоже что и `n/a`), но этот "пропуск" не значителен, так так он составляет всего 0.005% от общего количества данных в этом столбце. Для ответа на поставленные банком вопросы мы наверное не будем группировать клиентов по полу (это ведь не индикатор платежеспособности!), и поэтому странное значение 'XNA' можно игнорировать или удалить.

### Выводы насчёт `income_type` и `debt`<a id="income_type&debt_review"></a>

In [19]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [20]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

Ни в одном из этих столбцов нет пропусков, данные качественно оформлены что касается регистра букв в столбце `income_type`. Эти столбцы готовы к выводу зависимостей.

### Выводы насчёт `purpose`<a id="purpose_review"></a>

In [21]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Возможные причины одинаковых по смыслу значений и способ их замены:**

Здесь явно нужно будет провести лемматизацию, так как например значения 'свадьба', 'на проведение свадьбы' и 'сыграть свадьбу' обозначают одну и туже цель получения кредита.

Ошибки в данных скорее всего возникли из-за человеческого фактора - банку желательно предоставлять более подробные инструкции по заполнению анкеты своим клиентам (например, описывать цель получения кредита одним словом) или подготовить список с 3-5 вариантами ответов (жизненное мероприятие, недвижимость, ...), где бы клиенты просто обозначали общую категорию, к котой относится их конкретная цель.

## Предобработка данных<a id="datawork"></a>

### Обработка подозрительно высоких и отрицательных трудовых стажей<a id="datawork1"></a>

До любых расчётов трудового стажа отрицательные значения нужно заменить на положительные:

In [22]:
data['days_employed'] = data['days_employed'].abs()

# Этот-же метод применяем к столбцу 'years':
data['years'] = data['years'].abs()

# Проверяем, действительно ли мы избавились от отрицательных значений:
display(data.loc[data['days_employed'] < 0]['days_employed'].count())
display(data.loc[data['years'] < 0]['years'].count())

0

0

Ещё прежде чем преступить к замене пропусков на средние значения, нужно избавиться от слишком больших значений общего трудового стажа для всех категорий кроме 'безработных' и 'пенсионеров', иначе они могут повлиять на средние значения отдельных видов деятельности. Заменим их на `NaN`. Сейчас мы работаем со вспомогательным столбцом `years`, так как его значения удобнее сравнивать с числом 42 чем значения `total_employed` с 15330 днями максимального стажа.

In [23]:
# Проверим - сколько изначально пропусков:
data['years'].isnull().sum()

2174

In [24]:
# Прописываем метод для замены слишком больших стажей на 'NaN':
d = 0
while d < len(data['years']):
    if data.loc[d, 'years'] > 42:
        if data.loc[d, 'income_type'] != 'пенсионер' and data.loc[d, 'income_type'] != 'безработный':
            data.loc[d, 'years'] = math.nan
    d +=1

# Проверяем новое количество пропусков:
data['years'].isnull().sum()

2184

У нас всё получилось, к пропускам добавились 3 компаньона и 7 сотрудников со сверхвысокими стажеми, итого +10.

In [25]:
# Сейчас мы можем добавить эти значения в исходный столбец 'days_employed' и удалить доп. столбец 'years':
data['days_employed'] = data['years'] * 365
data = data.drop(['years'], axis=1)

# Проверяем пропуски:
data['days_employed'].isnull().sum()

2184

In [26]:
#median_days = data.groupby('income_type').agg({'days_employed': ['median']}).reset_index()
#median_days.columns = median_days.columns.droplevel(0)
#columns = ['income_type', 'median_days']
#median_days.columns = columns

#mean_days = data.groupby('income_type').agg({'days_employed': ['mean']}).reset_index()
#mean_days.columns = mean_days.columns.droplevel(0)
#columns = ['income_type', 'mean_days']
#mean_days.columns = columns

# Для удобного сравнения объединяем таблицы:
#merged_stat_days = mean_days.merge(median_days, on = 'income_type', how = 'left')
#merged_stat_days

In [27]:
# Тот-же процесс повторяем для дохода:

#median_income = data.groupby('income_type').agg({'total_income': ['median']}).reset_index()
#median_income.columns = median_income.columns.droplevel(0)
#columns = ['income_type', 'median_income']
#median_income.columns = columns

#mean_income = data.groupby('income_type').agg({'total_income': ['mean']}).reset_index()
#mean_income.columns = mean_income.columns.droplevel(0)
#columns = ['income_type', 'mean_income']
#mean_income.columns = columns

# Для удобного сравнения опять объединяем таблицы:
#merged_stat_income = mean_income.merge(median_income, on = 'income_type', how = 'left')
#merged_stat_income

Эти столбцы содержат количественные переменные, поэтому для заполнения пропусков используем характерные значения нашей выборки. Сначала находим среднее арифметическое и медиану для общего стажа, предварительно группируя клиентов по их виду деятельности и образованию способом формирования сводной таблицы, а потом склеивая таблицу по 'индексу' (`income_type`) с помощью `join()`.

Мы выбрали два критерия группировки данных, так как отработаный стаж зависит не только от вида деятельности, но и от времени потраченного на образование: кто-то может выйти на работу и стать сотрудником сразу после начального, а кто-то - отучиться ещё лет 5-10 и это скорее всего повлияет на карьерные возможности второго клиента. Поэтому среднее просто по виду деятельности является не самым лучшим показателем без контекста уровня образования.

In [28]:
# Для успешной группировки перепишем все значения в столбце 'education' строчными буквами, что бы нам всё равно пришлось сделать, чтобы качественно обработать данные.
data['education'] = data['education'].str.lower()

# Делаем сводные таблицы со средним и медианой общего трудового стажа и склеиваем их:
data_grouped_days_median = data.pivot_table(index = ['income_type'], columns = ['education'], values = 'days_employed', aggfunc = ['median'])
data_grouped_days_mean = data.pivot_table(index = ['income_type'], columns = ['education'], values = 'days_employed', aggfunc = ['mean'])
data_grouped_days = data_grouped_days_mean.join(data_grouped_days_median)

# Для последующего удобного обращения к столбцу вида деятельности добавим его вне индекса:
data_grouped_days['income_type'] = data_grouped_days.index
data_grouped_days.reset_index(drop = True)

mean                                                    \
education         высшее      начальное неоконченное высшее        среднее   
0          395302.838654            NaN                 NaN  337524.466835   
1                    NaN            NaN                 NaN    3296.759962   
2            3182.314561    3483.831436         2216.913600    3604.210297   
3            2018.687160    2019.310449         1412.927180    2219.874950   
4          365176.302579  362288.367962       369795.282176  365007.457531   
5             520.848083            NaN                 NaN            NaN   
6            2247.236790    1853.910264         1590.509656    2376.978668   
7             578.751554            NaN                 NaN            NaN   

                                 median                                     \
education ученая степень         высшее      начальное неоконченное высшее   
0                    NaN  395302.838654            NaN                 NaN   
1                    NaN            NaN            NaN                 NaN   
2            5968.075884    2531.034209    2787.767403         1885.183639   
3                    NaN    1454.659104    1151.634460         1017.888120   
4          356930.517546  366158.526428  360264.985350       372250.501660   
5                    NaN     520.848083            NaN                 NaN   
6            2704.223421    1553.815154    1197.176853         1197.676879   
7                    NaN     578.751554            NaN                 NaN   

                                             income_type  
education        среднее ученая степень                   
0          337524.466835            NaN      безработный  
1            3296.759962            NaN        в декрете  
2            2857.770974    5968.075884      госслужащий  
3            1668.570970            NaN        компаньон  
4          365025.338867  356930.517546        пенсионер  
5                    NaN            NaN  предприниматель  
6            1612.831545    2351.431934        сотрудник  
7                    NaN            NaN          студент

По результатам мы видим, что это был правильный ход - ведь стажи между клиентами с одним видом деятельности но разным образованием могут отличаться на 700-800 дней. В то же время мы видим много `NaN` - значит, таких данных просто нет и программе не из чего посчитать среднее значение. Это говорит о том, что **выборка не совсем репрезентативна**, так как какие-то типы клиентов здесь вообще отсутствуют, или же по ним отсуствуют данные (строка в таблице есть, но там стоит `NaN`). Именно такие значения нам прийдётся заполнять просто средним значением по данному виду деятельности невзирая на уровень образования, но с этим мы разберёмся по ходу ситуации, а сначала заменим их на `error`.

Что касается различия между медианой и средним арифметическим, значения среднего сильно завышены для некоторых видов деятельности (например, в случае стажей компаньонов), причём завышены данные для всех уровней образования. Значит, **выборка асимметрична** с крупными выбросами в нескольких категориях, кроме тех совершенно невероятных значений, которые мы уже удалили. Чтобы избежать влияния таких выбросов, будем использовать для замены **медиану**.

In [29]:
# Тот-же процесс повторяем для дохода:

data_grouped_income_median = data.pivot_table(index = ['income_type'], columns = ['education'], values = 'total_income', aggfunc = ['median'])
data_grouped_income_mean = data.pivot_table(index = ['income_type'], columns = ['education'], values = 'total_income', aggfunc = ['mean'])
data_grouped_income = data_grouped_income_mean.join(data_grouped_income_median)
data_grouped_income['income_type'] = data_grouped_income.index
data_grouped_income.reset_index(drop = True)

mean                                                    \
education         высшее      начальное неоконченное высшее        среднее   
0          202722.511368            NaN                 NaN   59956.991984   
1                    NaN            NaN                 NaN   53829.130729   
2          197320.548067  184056.353037       172476.953367  154055.103706   
3          242375.855503  165057.030695       197649.335647  179490.220250   
4          170667.987802  111314.924441       138312.108423  131698.933496   
5          499163.144947            NaN                 NaN            NaN   
6          191564.306210  137212.850187       174697.072430  152662.997144   
7           98201.625314            NaN                 NaN            NaN   

                                 median                                     \
education ученая степень         высшее      начальное неоконченное высшее   
0                    NaN  202722.511368            NaN                 NaN   
1                    NaN            NaN            NaN                 NaN   
2          111392.231107  172511.107016  148339.290825       160592.345303   
3                    NaN  201785.400018  136798.905143       179867.152890   
4          177088.845999  144240.768611  102598.653164       120136.896353   
5                    NaN  499163.144947            NaN                 NaN   
6          194310.337215  165640.744634  125994.910603       151308.937846   
7                    NaN   98201.625314            NaN                 NaN   

                                             income_type  
education        среднее ученая степень                   
0           59956.991984            NaN      безработный  
1           53829.130729            NaN        в декрете  
2          136652.970357  111392.231107      госслужащий  
3          159070.690289            NaN        компаньон  
4          114842.854099  177088.845999        пенсионер  
5                    NaN            NaN  предприниматель  
6          136555.108821  198570.757322        сотрудник  
7                    NaN            NaN          студент

Различия между людми из той-же сферы деятельности, но с разным образованием - существенны и достигают десятков тысяч, значит, решение использовать больше одного критерия для группировки данных оправдано.

Здесь мы тоже можем наблюдать пропуски и завышеные значения среднего, опять принимаем решение использовать для замены медиану.

Приступим к замене пропусков в обоих столбцах медианой в зависимости от вида деятельности **и** уровня образования:

In [30]:
# Составим функции для замены 'NaN' средними значениями, используем конструкцию 'try-except',
# так как может возникнуть ситуация, что для данной категории (комбинации данного вида деятельности и уроня образования) нет значений,
# по которым можно посчитать медиану. Тогда выводим 'Error'.
def replace_nan_days(data):
    if math.isnan(data[1]) == True:
        try:
            for m in range(len(data_grouped_days)):
                if data[8] == data_grouped_days['income_type'][m]:
                    return data_grouped_days['median'][data[3]][m]
        except:
            return 'Error'
    else:
        return data[1]
def replace_nan_income(data):
    if math.isnan(data[10]) == True:
        for m in range(len(data_grouped_income)):
            if data[8] == data_grouped_income['income_type'][m]:
                return data_grouped_income['median'][data[3]][m]
    else:
        return data[10]
data['days_employed'] = data.apply(replace_nan_days, axis = 1)     
data['total_income'] = data.apply(replace_nan_income, axis = 1)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Проверяем, действительно ли удалось заменить все пропуски и есть ли у нас какие-to `error`:

In [181]:
display(data['days_employed'].isnull().sum())
display(data['total_income'].isnull().sum())
display(data.loc[data['days_employed'] == 'Error'].count())
display(data.loc[data['total_income'] == 'Error'].count())

0

0

children                 0
days_employed            0
dob_years                0
education                0
education_id             0
family_status            0
family_status_id         0
gender                   0
income_type              0
debt                     0
total_income             0
purpose                  0
income_level_category    0
dtype: int64

children                 0
days_employed            0
dob_years                0
education                0
education_id             0
family_status            0
family_status_id         0
gender                   0
income_type              0
debt                     0
total_income             0
purpose                  0
income_level_category    0
dtype: int64

**Вывод:**

Нам удалось аккуратно заменить все пропуски и, по возможности, непрабдоподобные значения в столбцах `days_employed` и `total_income` на значения соответсвующие средним значениям, сгруппировав клиентов по виду деятельности и образованию. `Error` отсуствуют, значит, у нас не возникла ситуация, когда были пропущены все значения данной категории.


### Замена типа данных столбцов `days_employed` и `total_income`<a id="datawork3"></a>

Из первоначального обзора данных мы знаем, что в столбцах 'days_employed' и 'total_income' значения принимают формат 'float64' и это нужно перевести в 'int64', так как стаж обычно не считается с точностью до сотых дня, а доход скорее всего получают в целых рублях/долларах, значит копейки/центы нужно округлить.

In [32]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
# используем метод 'astype', а не 'to_numeric',
# так так 'to_numeric' возвращает данные типа 'float', а нам их нужно как раз перевести на целочисленный формат.

# проверяем замену типа данных, ещё раз вызывая info():
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Исправление значений в столбце с детьми<a id="datawork4"></a>

Мы уже говорили о том, что подозрительные данные можно либо заменить, либо удалить. Попробуем их заменить на значения '1' и '2', которые, наверное, имелись ввиду вместо '-1' и '20'. 

In [33]:
f = 0
while f < (len(data['children'])):
    if data.loc[f, 'children'] == 20:
        data.loc[f, 'children'] = 2
    if data.loc[f, 'children'] == -1:
        data.loc[f, 'children'] = 1
    f += 1
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод:**

Странных значений больше нет, столбец готов к поиску закономерностей. Продолжаем обрабатывать оставшиеся проблемы в данных.

### Работа с нулевым значением возраста<a id="datawork5"></a>

Нулевые значения возраста можно удалить или заменить на среднее значение для данного вида деятелности, по принципу уже применнённому к общему трудовому стажу используя 2 критерия группировки данных для нахождения среднего значения возраста: вид деятельности и уровень образования. Выбираем второй вариант - замену, следуя логичному предположению, что например все студенты и пенсионеры с одним уровнем образования принадлежат +/- к одной возрастной категории, которую можно описать средним значением.

In [35]:
# Начинаем с замены значений "0" на "NaN", иначе они могут испортить расчёт среднего:
def makenan(value):
    if value == 0:
        value = math.nan
        return value
    else:
        return value
data['dob_years'] = data['dob_years'].apply(makenan)
data['dob_years'].isnull().value_counts()

False    21424
True       101
Name: dob_years, dtype: int64

Сейчас мы имеем 101 пропуск заместо нашего 101 нулевого значения. Продалжаем расчёт средних значений по типу деятельности и уровню образования, используем код, похожий на способ сортировки данных выше.

In [36]:
age_stat_median = data.pivot_table(index = ['income_type'], columns = ['education'], values = 'dob_years', aggfunc = ['median'])
age_stat_mean = data.pivot_table(index = ['income_type'], columns = ['education'], values = 'dob_years', aggfunc = ['mean'])
age_stat = age_stat_mean.join(age_stat_median)
age_stat['income_type'] = age_stat.index
age_stat.reset_index(drop = True)

mean                                                           \
education     высшее  начальное неоконченное высшее    среднее ученая степень   
0          45.000000        NaN                 NaN  31.000000            NaN   
1                NaN        NaN                 NaN  39.000000            NaN   
2          39.188645  46.125000           30.884615  42.412530      36.000000   
3          38.913337  37.666667           33.227758  41.063487            NaN   
4          59.101576  61.020202           59.463415  59.362792      65.500000   
5          42.500000        NaN                 NaN        NaN            NaN   
6          38.448217  40.751724           33.714674  40.730446      46.666667   
7          22.000000        NaN                 NaN        NaN            NaN   

          median                                                       \
education высшее начальное неоконченное высшее среднее ученая степень   
0           45.0       NaN                 NaN    31.0            NaN   
1            NaN       NaN                 NaN    39.0            NaN   
2           37.0      43.5                30.5    42.0           36.0   
3           38.0      36.5                31.0    41.0            NaN   
4           59.0      61.0                59.0    60.0           65.5   
5           42.5       NaN                 NaN     NaN            NaN   
6           37.0      39.0                31.0    40.0           45.0   
7           22.0       NaN                 NaN     NaN            NaN   

               income_type  
education                   
0              безработный  
1                в декрете  
2              госслужащий  
3                компаньон  
4                пенсионер  
5          предприниматель  
6                сотрудник  
7                  студент

Итак, между средними возрастами для разных уровней образования есть различия аж на 5-7 лет. Медиана ниже среднего у некоторых значений, но отличия не слишком значительны и мы можем использовать среднее арифметичесткое для замены. Пишем функцию замены, похожую на те, которые мы уже применяли для столбцов `days_employed` и `total_income`:

In [39]:
# Приступаем к замене нулевых значений:
def replace_nan_age(data):
    if math.isnan(data[2]) == True:
        # Опять пользуемся конструкцией 'try-except' на случай,
        # если для данной комбинации уровня образования и видя деятельности не из чего посчитать среднее значение возраста.
        try:
            for m in range(len(age_stat)):
                if data[8] == age_stat['income_type'][m]:
                    return age_stat['median'][data[3]][m]
        except:
            return 'Error'
    else:
        return data[2]
data['dob_years'] = data.apply(replace_nan_age, axis = 1)
# Проверяем замену и наличие 'Errors':
display(data.loc[data['dob_years'] == 0]['dob_years'].count())
display(data['dob_years'].isnull().sum())
display(data.loc[data['dob_years'] == 'Error'].count())


0

0

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Из-за замены на `NaN`, тип данных в этом столбце поменялся на `float`, поэтому мы используем `astype()` чтобы вернуть его к формату `int`:

In [40]:
data['dob_years'] = data['dob_years'].astype('int')

# Проверяем, все-ли типы данных соответсвуют 'int' или 'object' ('str'):
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


#### Разбираемся с нюансом в `dob_years`, непредвиденным до замены нулевых значений <a id="datawork5.1"></a>


Казалось - столбец готов, но при нашем изначальном поиске "странных значений" нам мешали нули и мы не могли обнаружить другие слишком маленькие значения с помощью `min()`. Сейчас давайте сгруппируем столбец по виду деятельности, а к значениям возраста применим `min()`, `max()` и `mean()`:

In [41]:
data.groupby('income_type').agg({'dob_years': ['min','max', 'mean']})

dob_years               
                      min max       mean
income_type                             
безработный            31  45  38.000000
в декрете              39  39  39.000000
госслужащий            19  75  40.795751
компаньон              19  74  39.852311
пенсионер              22  74  59.372666
предприниматель        27  58  42.500000
сотрудник              19  74  40.013490
студент                22  22  22.000000

Всё выглядит достаточно логично кроме 22-летних пенсионеров, с этим надо разобраться. Создадим одельную таблицу с возрастами пенсионеров:



In [42]:
pensioners = pd.DataFrame(columns=['count'])
pensioners['count'] = data.loc[data['income_type'] == 'пенсионер']['dob_years'].value_counts()
pensioners = pensioners.sort_index(ascending = True)
pensioners.reset_index(drop = True)

,count
0,1
1,1
2,2
3,3
4,1
5,1
6,3
7,2
8,3
9,1


Визуально понятно, что большенство пенсионеров приходятся на средний и пожилой возраст и этого стоило ожидать, но есть и уникумы которым по 22, 24, 32 года... Учитывая особые ситуации рабочих на Крайнем Севере, в тяжёлых условиях итд., допустим что общеприемлемый пенсионный возраст - 40 лет. Найдём число 'слишком молодых пенсионеров' и их долю по отношению к общему количеству клиентов на пенсии, а потом будем решать, слишком ли много таких странных случаев чтобы с ними разбираться (удалять, заменять):

In [43]:
print('Число достаточно пожилых пенсионеров: {}'.format(pensioners.loc[40:]['count'].sum()))
print('Число слишком молодых пенсионеров: {}'.format(pensioners.loc[:39]['count'].sum()))
print('Доля слишком молодых пенсионеров от их общего числа: {:.2%}'.format(pensioners.loc[:39]['count'].sum() / pensioners['count'].sum()))

Число достаточно пожилых пенсионеров: 3814
Число слишком молодых пенсионеров: 42
Доля слишком молодых пенсионеров от их общего числа: 1.09%


Всего 1% - это допустимо, возможно, эти люди - инвалиды и поэтому получают пенсию не смотря на свою молодость. Таким образом данные не в коем случае не испорчены и эти значения можно оставить, так как такая малая доля инвалидов или рабочих с особыми случаями объяснима, удалять данные лишний раз не стоит, а в замене мы не можем быть уверены без дополнительных данных от банка - может эти люди и правда на пенсии в 20-30 лет или минимальный общеприемлемый пенсионный возраст не 40, а 35 или 45. Важно заметить, что до этого мы заменяли нулевые значения средним арифметическим, которое было расчитано включая и эти маленькие значения. Это оправдуемо, так как мы и считали среднее арифметическое и медиану по этой конкретной выборке, а не выбирали значения 'среднего возраста', например, всех пенсионеров в РФ.

**Вывод:**

Нулевых или пропущеных значений больше нет, данные в формате `int`, мы имеем ввиду слишком низкий возраст пенсионеров и мы объяснили это небольшое количество странных сначений, итого: столбец готов к поиску закономерностей. Продолжаем обрабатывать оставшиеся проблемы в данных.

### Замена больших букв<a id="datawork6"></a>

Чтобы было удобно обрабатывать столбец с семейным положением клиентов, перепишем все значения маленькими буквами:

In [44]:
data['family_status'] = data['family_status'].str.lower()

# Проверяем уникальные значения в 'family_status':
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

По аналогичному принципу нужно применить `lower()` к `education`, но мы это уже сделали в процессе группировки данных для обработки значений в `days_employed` и  `total income`. Поэтому код закомментирован - он нам уже не нужен, но мы можем проверить, действительно ли количество и распределение значений в `education` соответствует `education_id` и все они написаны строчными буквами.

In [45]:
# data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [46]:
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

**Вывод:**

Сейчас мы можем сопоставить количества уникальных значений идентификаторов уровня образования и количества уникальных значений названий уровня образования - они совпадают, значет мы всё сделали верно.

### Удаление значения 'XNA' в столбце с полом клиентов<a id="datawork7"></a>

Так как мы не можем узнать пол клиента `XNA` и это значение составляет всего 0.005% от всех данных, мы можем его удалить:

In [47]:
data = data[data.gender != 'XNA']
data['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

**Вывод:**

Странных неоприделённых значений больше нет, данные столбца готовы к анализу.

### Обработка дубликатов<a id="datawork8"></a>

#### Автоматизированое удаление дубликатов<a id="datawork81"></a>

Важное разъяснение: не смотря на нашу замену некоторых ячеек средними статистическими данными, количество дубликатов не должно внезапно увеличиться, так как строки всё равно будут отличатся по знячениям других столбцов. А если строки совпадают после замены `NaN` средним арифметическим, то они бы совпадали и если бы мы `NaN` не заменяли.

Дубликаты, скорее всего, возникли из-за неаккуратного формирования файла банком - возможно данные переносились из анкет в базу данных вручную и один клиент туда попал два раза, также мог произойти сбой и некоторые данные могли задублироваться. 

In [48]:
# Проверяем наличие дубликатов:
data.duplicated().sum()

71

In [49]:
# Эти 71 повторных значения нужно удалить!
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

0

**Вывод:**

Все дубликаты, которые мы можем выявить автоматически, удалены, продолжим их поиск отдельно в столбце `purpose`. 

#### Ручной просмотр дубликатов в столбце `purpose`, определение основных значений с помощью подсчёта  `lemmas`<a id="datawork82"></a>

In [50]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка своего жилья                      620
покупка недвижимости                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Проанализировав все возможные цели получения кредитов, мы выявили 4 основных цели: 'свадьба', 'автомобиль', 'недвижимость', 'образование'. Потвердим их с помощью подсчёта `lemmas`:

In [51]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
lemmlist = []
for purpose in data['purpose']:
    lemma1 = m.lemmatize(purpose)
    lemmlist.extend(lemma1)
display(Counter(lemmlist))

Counter({'покупка': 5896,
         ' ': 33569,
         'жилье': 4460,
         '\n': 21453,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6350,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

**Вывод:**

Кроме пояснительных слов, основными целями действительно являются нами выявленные 'свадьба', 'автомобиль', 'недвижимость' (или жильё - объеденим их в одну категорию), и 'образование'. Далее проведём упрощение данных в столбце `purpose`.


### Лемматизация<a id="lemmas"></a>

Для замены подобных значений создаём метод для проведения лемматизации и упрощения данных в столбце `purpose`. Важно заметить, что мы используем конструкцию `try-except` на случай если мы ошиблись и в столбце есть значения, которые не имеют ничего общего со 'свадьбой', 'автомобилем', 'недвиюимостью' или 'образованием'. Тогда в столбце появится `error`.

In [52]:
u = 0
while u < len(data['purpose']):
    purpose = data.loc[u, 'purpose']
    lemmas = m.lemmatize(purpose)
    for word in lemmas:
        try:
            if 'свадьба' in word:
                data.loc[u, 'purpose'] = 'свадьба'
            if 'автомобиль' in word:
                data.loc[u, 'purpose'] = 'автомобиль'
            if 'недвижимость' in word:
                data.loc[u, 'purpose'] = 'недвижимость'
            if 'жилье' in word:
                data.loc[u, 'purpose'] = 'недвижимость'
            if 'образование' in word:
                data.loc[u, 'purpose'] = 'образование'
        except:
            print('error')
    u += 1

# Проверим количество значений 'purpose':
data['purpose'].value_counts()


недвижимость    10810
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64

**Вывод**

Нет никаких значений `error`, лемматизация данных и упрощение столбца `purpose` прошли успешно. Двигаемся дальше.

### Категоризация данных<a id="categorisation"></a>

Для ответа на один из поставленых нам вопросов ('Есть ли зависимость между уровнем дохода и возвратом кредита в срок?') нам нужно категоризировать данные по уровню дохода, иначе сводная таблица получится слишком длинной. Сделаем это с помощью нахождения квартилей - так мы получим 4 одинаковые по количеству данных репрезентативные категории, чьи свойства (например способность оплаты кредитов всрок) мы можем в дальнейшем сравнивать.

In [53]:
income_level = data['total_income'].quantile([0.25, 0.5, 0.75]).astype('int')
display(income_level)
# Это разделяющие значения, по ним мы будем сортировать данные столбца 'total_income'

0.25    107515
0.50    143707
0.75    198301
Name: total_income, dtype: int64

Итак, нижние 25% нашей выборки имеют доход меньше 107620 (Q1), их категорию дохода мы можем назвать `низкий доход`. Клиенты следующего квартиля входят в срез от 25% до 50% и имеют доход от 107620 (Q1) до 151876 (Q2). Их категорию назовём `средний доход`. Следующие 25% клиентов имеют доход от 151876 (Q2) до 202417 (Q3), это люди, у которых `доход выше среднего`. Верхние 25% будут иметь `сверхвысокий доход`, от 202417 (Q3) и до максимального значения в нашей выборке. Расстояния Q2 - Q1 и Q3 - Q2 почти одинаковы (около 50 тысяч), это говорит о том, что наша выборка почти симметрична. Конечно, возможно в реальном мире и 107620 - совсем не низкий доход, но не имея контекста и данных о 'генеральной совокупности', мы можем использовать метод категоризации с помощью квантилей.

Приступим к разделению клиентов:

У нас получилось не совсем равномерное распределение, видимо из-за того, что несколько елементов принимают значение 202417. Попробуем устранить проблему "в ручную", модифицировав метод.

In [54]:
# Напишем метод для категоризации клиентов по их уровню дохода, выводам присвоим новый столбец в 'data':
# Название нового столбца 'income_level_category' = категория уровня дохода
def sort_by_income(element):
    if element < 107620:
        return 'низкий доход'
    if 107620 <= element < 151876:
        return 'средний доход'
    if 151876 <= element < 202417:
        return 'доход выше среднего'
    if element >= 202417:
        return 'сверхвысокий доход'
data['income_level_category'] = data['total_income'].apply(sort_by_income)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_level_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,сверхвысокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,сверхвысокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,доход выше среднего
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость,сверхвысокий доход
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость,сверхвысокий доход
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,средний доход
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,низкий доход
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость,средний доход


**Вывод:**

Сейчас у нас данные категоризированы по уровню дохода и мы можем приступать к финальному анализу и поиску зависимостей..


## Подведение итогов, ответы на вопросы:<a id="results"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [55]:
# Посчитаем количество должников, сгруппировав их по количеству детей:
data_pivot1 = data.pivot_table(index='children', values='debt', aggfunc = 'sum').reset_index()
columns = ['children', 'debt']
data_pivot1.columns = columns

# Добавим общее количество людей в выборке, сгруппировав их по количеству детей:
data_pivot1['total_clients'] = data.groupby('children').agg({'debt': ['count']})

# Находим соотношение должников к общему количеству клиентов, сгруппировав их по количеству детей.
# Ответ округляем до сотых и выводим со знаком %:
data_pivot1['ratio'] = (data_pivot1['debt'] / data_pivot1['total_clients'] * 100).round(2).astype('str') + '%'
data_pivot1.sort_values(by = 'ratio', ascending = False)

,children,debt,total_clients,ratio
4,4,4,41,9.76%
2,2,202,2128,9.49%
1,1,445,4855,9.17%
3,3,27,330,8.18%
0,0,1063,14090,7.54%
5,5,0,9,0.0%


**Вывод:**

Да, в числе клиентов с детьми пропорционально больше должников чем числе бездетных клиентов, исключением являются клиенты с 5 детьми. Скорее всего за этим стоит либо не репрезентативность выборки (в базе банка всего 9 таких клиентов), либо бо́льшая финансовая ответсвенность клиентов с такими многодетными семьями (но это менее вероятно, так как для клиентов с 4 детьми ситуация состоит с точностью наоборот).




- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [56]:
# Посчитаем количество должников, сгруппировав их по семейному положению:
data_pivot2 = data.pivot_table(index='family_status', values='debt', aggfunc = 'sum').reset_index()
columns = ['family_status', 'debt']
data_pivot2.columns = columns

# Добавим общее количество людей в выборке, сгруппировав их по семейному положению:
#data_pivot2['total_clients'] = 
total_clients_by_family = data.groupby('family_status').agg({'debt': ['count']}).reset_index()
total_clients_by_family.columns = ['family_status', 'total_clients']

data_pivot2['total_clients'] = total_clients_by_family['total_clients']

# Находим соотношение должников к общему количеству клиентов, сгруппировав их по семейному положению.
# Ответ округляем до сотых и выводим со знаком %:
data_pivot2['ratio'] = (data_pivot2['debt'] / data_pivot2['total_clients'] * 100).round(2).astype('str') + '%'
data_pivot2.sort_values(by = 'ratio', ascending = False)

,family_status,debt,total_clients,ratio
4,не женат / не замужем,274,2810,9.75%
2,гражданский брак,388,4150,9.35%
3,женат / замужем,931,12339,7.55%
0,в разводе,85,1195,7.11%
1,вдовец / вдова,63,959,6.57%


**Вывод:**

Да, клиенты, которые никогда не состояли в официальном браке, склонны к задолженности по возврату кредитов. Видимо, они менее ответственны или не могут положиться на финансовую поддержку членов семьи, к которой их обязывают формальные связи.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [57]:
# Посчитаем количество должников, сгруппировав их по количеству детей:
data_pivot3 = data.pivot_table(index='income_level_category', values='debt', aggfunc = 'sum').reset_index()
columns = ['income_level_category', 'debt']
data_pivot3.columns = columns

# Добавим общее количество людей в выборке, сгруппировав их по количеству детей:
total_clients_by_income = data.groupby('income_level_category').agg({'debt': ['count']}).reset_index()
total_clients_by_income.columns = ['income_level_category', 'total_clients']
data_pivot3['total_clients'] = total_clients_by_income['total_clients']

# Находим соотношение должников к общему количеству клиентов, сгруппировав их по количеству детей.
# Ответ округляем до сотых и выводим со знаком %:
data_pivot3['ratio'] = (data_pivot3['debt'] / data_pivot3['total_clients'] * 100).round(2).astype('str') + '%'
data_pivot3.sort_values(by = 'ratio', ascending = False)

,income_level_category,debt,total_clients,ratio
3,средний доход,568,6275,9.05%
0,доход выше среднего,401,4901,8.18%
1,низкий доход,427,5369,7.95%
2,сверхвысокий доход,345,4908,7.03%


**Вывод:**

Да, зависимость есть - клиенты средних 50% нашей выборки наиболее склонны к задолженности (и между % должников в этих категориях наименьшее различее). Видимо, люди с низким доходом вынуждены жить более расчётливо, а для людей со сверхвысоким доходом не проблема оплатить кредиты в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [58]:
data_pivot4 = data.pivot_table(index = 'purpose', values = 'debt', aggfunc = 'sum').reset_index()
columns = ['purpose', 'debt']
data_pivot4.columns = columns

# Добавим общее количество людей в выборке, сгруппировав их по количеству детей:
total_clients_by_purpose = data.groupby('purpose').agg({'debt': ['count']}).reset_index()
total_clients_by_purpose.columns = ['purpose', 'total_clients']
data_pivot4['total_clients'] = total_clients_by_purpose['total_clients']

# Находим соотношение должников к общему количеству клиентов, сгруппировав их по количеству детей.
# Ответ округляем до сотых и выводим со знаком %:
data_pivot4['ratio'] = (data_pivot4['debt'] / data_pivot4['total_clients'] * 100).round(2).astype('str') + '%'
data_pivot4.sort_values(by = 'ratio', ascending = False)

,purpose,debt,total_clients,ratio
0,автомобиль,403,4306,9.36%
2,образование,370,4013,9.22%
3,свадьба,186,2324,8.0%
1,недвижимость,782,10810,7.23%


**Вывод:**

Да, самый большой процент должников мы находим среди клиентов, берущих кредит для покупки машины, видимо это менее обдуманная инвестиция чем образование или свадьба. Образование обычно окупается спустя длинный промежуток времени, а свадьбы не окупаются вообще, но здесь возможно стоимость кредитов не так велика, или в их погашении помогают молодожёнам родственники. Недвижимость - наоборот, наиболее осознаная причина взять у банка кредит, такие клиенты наиболее ответственны, а также возможно имеют доход с недвижимости (аренда, коммерческое использование), что им помогает гасить долги в срок.


## Заключение - словари в данных:<a id="dictionaries"></a>

В процессе исследования мы обнапужили два ключевых словаря: 

`education -> education_id`

`family_status -> family_status_id`

В обоих словарях качественные данные (названия уровня образования и семейного положения) сопоставлены с количественными идентификаторами. Это нам помогло при обработке данных, так как при правильной замене букв на строчные мы получили одинаковые количества, например, количество людей с оконченым вышшим образованием равно 5250, что равно и количеству людей с `education_id` номером `0`. Мы можем составить сводные таблицы и определить, какой индекс соответсвует какому названию уровня образования:

In [182]:
data['education'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'name'}).merge(
    data['education_id'].value_counts().to_frame(), on = data['education'].value_counts(), right_index=True
).drop('key_0', axis = 1).merge(
    data['education_id'].value_counts().to_frame().reset_index()
).rename(columns={'index' : 'id'})

,name,education,education_id,id
0,среднее,15172,15172,1
1,высшее,5250,5250,0
2,неоконченное высшее,743,743,2
3,начальное,282,282,3
4,ученая степень,6,6,4


In [183]:
# И повторить тот-же процесс для 'family_status' и 'family_status_id':
data['family_status'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'name'}).merge(
    data['family_status_id'].value_counts().to_frame(), on = data['family_status'].value_counts(), right_index=True
).drop('key_0', axis = 1).merge(
    data['family_status_id'].value_counts().to_frame().reset_index()
).rename(columns={'index' : 'id'})

,name,family_status,family_status_id,id
0,женат / замужем,12339,12339,0
1,гражданский брак,4150,4150,1
2,не женат / не замужем,2810,2810,4
3,в разводе,1195,1195,3
4,вдовец / вдова,959,959,2


Все цифры совпадают, обработка данных проведена правильно и сейчас легко сопоставить "буквенное" название с числовым. В дальнейшем можно использовать идентификатор той или иной категории, если нужно будет проводить математические операции с её значениями - свойство иммено количественных данных, а не названий.



## Общий вывод<a id="conclusion"></a>


Наименее надёжными для банка являются клиенты у которых есть дети, среди них наибольшая доля невовремя возвращённых кредитов приходится на клиентов с 4 детьми - 9.76%. Клиенты со средним доходом, которые вписываются между Q1 и Q3 выборки, также ненадёжны - здесь процент задолженности 9.05% для людей со средним доходом и 8.18% для людей с доходом выше среднего. Что касается семейного положения, клиенты, которые не состоят (и никогда не состояли) в официальном браке - наибольшие должники, их доля 9.75% в категории не женатых/не замужем и 9.35% среди тех, кто состоит в гражданском браке. Наименее надёжная цель кредита - покупка автомобиля, целых 9.36% клиентов с кредитом, взятым на манипуляции с авто, его не вернули в срок.

Что касается самой выборки и данных, стоит заметить, что некоторых значений слишком мало для репрезентативного анализа, как например количество клиентов с 5 детьми или количество предпринимателей. Для этого мы можем порекоммендовать увеличить выборку и, по возможности, включить все значимые для анализа категории (количество детей - как раз одна из них) в равной мере. Также можно улучшить качество данных: например, можно создать варианты ответов для значений 'цели кредита' и в дальнейшем предлогать клиентам выбирать из ограниченого количества целевых категорий. Таким образом процесс обработки данных ускорится и не надо будет проводить лемматизацию.



## Чек-лист готовности проекта<a id="checklist"></a>

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.